### 1.修改区域颜色

In [66]:
import cv2
import numpy as np
print(cv2.__version__)

4.11.0


In [67]:
img = cv2.imread('jupyter/lenacolor.png')
img[0,0, 0:3]

array([125, 137, 226], dtype=uint8)

In [68]:
img1 = img.copy()
img1[0:50, 0:100, :] = [255, 255, 255]
img1[50:100, 0:100, :] = [128, 128, 128]
img1[100:150, 0:100, :] = [0, 0, 0]
img1[150:200, 0:100, :] = [0, 0, 255]
display(img1[0, 0, 0:3])
# cv2.imshow('img1', img1)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
cv2.imwrite('result/task1/modified_lenacolor.png', img1)

array([255, 255, 255], dtype=uint8)

True

### 2. 读取彩色图片并创建掩码

In [69]:
img = cv2.imread('jupyter/x.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# mask = np.zeros(img.shape, dtype=np.uint8)
mask = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]
cv2.imwrite("result/task2/mask.png", mask)
cv2.imwrite('result/task2/gray_x.jpg', img)

True

### 3.形态学操作

In [70]:
img = cv2.imread("jupyter/4.jpg")

img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
cv2.imwrite("result/task3/open_4.jpg", opening)

tophat = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, kernel)
cv2.imwrite("result/task3/tophat_4.jpg", tophat)


img = cv2.imread("jupyter/5.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gradient = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel)
cv2.imwrite("result/task3/gradient_5.jpg",gradient)


img = cv2.imread("jupyter/6.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
close = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
cv2.imwrite("result/task3/close_6.jpg", close)

blackhat = cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, kernel)
cv2.imwrite("result/task3/blackhat_6.jpg", blackhat)



True

### 4.图像透视变换与 OCR 文字识别

In [1]:
# 导入工具包
import numpy as np
import argparse
import cv2
import os
import pytesseract
from PIL import Image


def order_points(pts):
	# 一共4个坐标点
	rect = np.zeros((4, 2), dtype = "float32")

	# 按顺序找到对应坐标0123分别是 左上，右上，右下，左下
	# 计算左上，右下
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]

	# 计算右上和左下
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]

	return rect

def four_point_transform(image, pts):
	# 获取输入坐标点
	rect = order_points(pts)
	(tl, tr, br, bl) = rect

	# 计算输入的w和h值
	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))

	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))

	# 变换后对应坐标位置
	dst = np.array([
		[0, 0],
		[maxWidth - 1, 0],
		[maxWidth - 1, maxHeight - 1],
		[0, maxHeight - 1]], dtype = "float32")

	# 计算变换矩阵
	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

	# 返回变换后结果
	return warped

def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
	dim = None
	(h, w) = image.shape[:2]
	if width is None and height is None:
		return image
	if width is None:
		r = height / float(h)
		dim = (int(w * r), height)
	else:
		r = width / float(w)
		dim = (width, int(h * r))
	resized = cv2.resize(image, dim, interpolation=inter)
	return resized

# 读取输入
image = cv2.imread("jupyter/receipt.jpg")
#坐标也会相同变化
ratio = image.shape[0] / 500.0
orig = image.copy()


image = resize(orig, height = 500)

# 预处理
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)

# 展示预处理结果
print("STEP 1: 边缘检测")
cv2.imshow("Image", image)
cv2.imshow("Edged", edged)
cv2.imwrite("result/task4/edged.jpg", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 轮廓检测
cnts, _ = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

# 遍历轮廓
for c in cnts:
	# 计算轮廓近似
	peri = cv2.arcLength(c, True)
	# C表示输入的点集
	# epsilon表示从原始轮廓到近似轮廓的最大距离，它是一个准确度参数
	# True表示封闭的
	approx = cv2.approxPolyDP(c, 0.02 * peri, True)

	# 4个点的时候就拿出来
	if len(approx) == 4:
		screenCnt = approx
		break

# 展示结果
print("STEP 2: 获取轮廓")
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv2.imshow("Outline", image)
cv2.imwrite("result/task4/outline.jpg", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 透视变换
warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)

# 二值处理
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
ref = cv2.threshold(warped, 100, 255, cv2.THRESH_BINARY)[1]
cv2.imwrite('result/task4/scan.jpg', ref)

# 展示结果
print("STEP 3: 变换")
cv2.imshow("Original", resize(orig, height = 650))
cv2.imshow("Scanned", resize(ref, height = 650))
cv2.imwrite("result/task4/orig.jpg", orig)
cv2.imwrite("result/task4/ref.jpg", ref)
cv2.waitKey(0)

# 文字提取
print("STEP 4: 文字提取")
# 对图像进行额外的预处理以提高文字识别效果
kernel = np.ones((2,2), np.uint8)
ref = cv2.dilate(ref, kernel, iterations=1)
ref = cv2.erode(ref, kernel, iterations=1)

# 将OpenCV图像转换为PIL图像
pil_image = Image.fromarray(ref)

# 使用pytesseract提取文字
text = pytesseract.image_to_string(pil_image, lang='chi_sim+eng')

# 将提取的文字保存到文件
with open('result/task4/extracted_text.txt', 'w', encoding='utf-8') as f:
    f.write(text)

print("提取的文字内容：")
print(text)
print("\n文字已保存到 result/task4/extracted_text.txt")

                 


STEP 1: 边缘检测


2025-03-25 19:31:54.479 Python[34638:894479] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-25 19:31:54.479 Python[34638:894479] +[IMKInputSession subclass]: chose IMKInputSession_Modern


STEP 2: 获取轮廓
STEP 3: 变换
STEP 4: 文字提取
提取的文字内容：
x oe Ke KR Re KR KK KR Ke KH KK

WH
FOODS

WHOLE FOODS MARKET

399 POST RD WEST

36u
365
365

365

BROTH

OLE
DS

WESTPORT, CT 06880
(203) 227-6858

BACUN
BACON
BACON
BACUN

Vv

mw WH

re

CHIC

FLOUR ALMUND
CHKN BRST BNLSS SK
HEAVY CREAM
BALSMC REDUCT

BEEF

DOCS

GRND
JUICE COF CASHEW
PINT ORGAHIE

85/15

HNY ALMOND BuiTeR

关关关头 TAX

.00

BAL

4 99
4.99
4.99
4 99
2.15
1.99
8.80
3.39
6.49
5.04
8.99
14.49

9.99
101.33

Tr

cae a)     T

manna 4

v"T

Tn TT


文字已保存到 result/task4/extracted_text.txt


: 

### 5.多人脸检测

In [72]:
import cv2

image = cv2.imread('jupyter/manyPeople.jpg') 
# 将彩色图像转换为灰度图像
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 加载面部检测分类器
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')  # 加载预训练的人脸检测分类器

# 加载眼睛检测分类器
eyeCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')  # 加载预训练的眼睛检测分类器

# 检测灰度图像中的人脸
faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

print("发现{0}张人脸!".format(len(faces)))  # 打印检测到的人脸数量
print("其位置分别是：")
print(faces)  # 打印人脸的坐标和大小信息（x, y, w, h）

for (x, y, w, h) in faces:  # 遍历检测到的每张人脸
    # 在图像上绘制绿色的人脸矩形框
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)


    # 检测眼睛区域
    eyes = eyeCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # 在人脸区域绘制蓝色的眼睛矩形框
    for (ex, ey, ew, eh) in eyes:
        cv2.rectangle(image, (ex, ey), (ex + ew, ey + eh), (0, 0, 255), 2)

# 保存带有人脸和眼睛检测框的图像
cv2.imwrite("result/task5/face_result.jpg", image)

发现11张人脸!
其位置分别是：
[[  97   99  169  169]
 [1147  104  160  160]
 [ 843  200   70   70]
 [ 888  104  167  167]
 [ 361   96  166  166]
 [ 623  101  162  162]
 [  99  375  160  160]
 [ 884  373  161  161]
 [1142  369  167  167]
 [ 357  373  165  165]
 [ 626  383  151  151]]


True

### 6.基于 MeanShift 的目标跟踪

In [73]:
import cv2 as cv 
import numpy as np

# 打开视频文件
cap = cv.VideoCapture("jupyter/video.mp4")

# 设置初始跟踪窗口 (x, y, w, h)
track_window = (800, 400, 200, 160)  
ret, frame = cap.read()  # 读取视频的第一帧，ret 表示是否成功，frame 是图像数据
x, y, w, h = track_window  

# 提取初始区域并转换为 HSV 颜色空间
hsv_roi = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

mask = cv.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))  # 创建掩码，过滤特定颜色范围

# 计算目标区域的色调直方图并归一化
roi_hist = cv.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX)

# 设置 MeanShift 终止条件为最大 10 次迭代或窗口移动小于 1 像素
termcriteria = (cv.TERM_CRITERIA_COUNT | cv.TERM_CRITERIA_EPS, 10, 1)


# 初始化视频写入对象
fourcc = cv.VideoWriter_fourcc(*'mp4v')  # 定义视频编码格式为 MP4V
out = cv.VideoWriter("result/task6/tracked_video.mp4", fourcc, 30.0, (frame.shape[1], frame.shape[0]))  # 创建输出视频，帧率 30，尺寸与输入帧相同

# 视频帧循环处理
while True:
    ret, frame = cap.read()  # 读取视频的下一帧
    if ret:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)  # 将当前帧转换为 HSV 颜色空间
        # 根据目标直方图计算反投影图像
        dst = cv.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

        # 应用 MeanShift 跟踪
        ret, track_window = cv.meanShift(dst, track_window, termcriteria)
        
        x, y, w, h = track_window  # 获取更新后的窗口坐标和大小
        img2 = cv.rectangle(frame, (x, y), (x+w, y+h), 255, 2)  # 在当前帧上绘制白色跟踪框，线宽为 2
        out.write(img2)  # 将带跟踪框的帧写入输出视频
    else:
        break  # 如果帧读取失败（视频结束），退出循环

# 释放资源
cap.release() 
out.release()

### 7.改进

In [ ]:
import numpy as np
import dlib
import cv2

def main():
	# 打开视频文件
	vs = cv2.VideoCapture("jupyter/video.mp4")

	# 获取视频的第一帧
	ret, first_frame = vs.read()
	if ret:
		# 设置初始追踪区域（使用原始坐标，不进行缩放）
		x, y, w, h = 800, 400, 200, 160
		
		# 获取原始视频尺寸
		(height, width) = first_frame.shape[:2]
		
		# 创建dlib追踪器
		tracker = dlib.correlation_tracker()
		rect = dlib.rectangle(x, y, x + w, y + h)
		rgb = cv2.cvtColor(first_frame, cv2.COLOR_BGR2RGB)
		tracker.start_track(rgb, rect)
		
		# 设置输出视频（使用原始尺寸）
		fourcc = cv2.VideoWriter_fourcc(*'mp4v')
		out = cv2.VideoWriter("result/task7/dlib_tracked.mp4", fourcc, 30.0, (width, height))

		# 视频流
		while True:
			# 读取当前帧
			ret, frame = vs.read()
			if not ret:
				break
				
			# 转换颜色空间用于追踪
			rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
			
			# 更新追踪器
			tracker.update(rgb)
			pos = tracker.get_position()
			
			# 获取追踪框坐标
			startX = int(pos.left())
			startY = int(pos.top())
			endX = int(pos.right())
			endY = int(pos.bottom())
			
			# 绘制追踪框
			cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
			cv2.putText(frame, "Tracking", (startX, startY - 10), 
				cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
			
			# 写入输出视频
			out.write(frame)
			
			# 按ESC退出
			if cv2.waitKey(100) & 0xFF == 27:
				break

	# 释放资源
	vs.release()
	out.release()
	cv2.destroyAllWindows()

if __name__ == "__main__":
	main()

### 8. 车道线检测

In [78]:
import numpy as np
import cv2 as cv 

# 灰度图转换
def grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
# Canny 边缘检测
def canny(image, low_threshold, high_threshold):
    return cv2.Canny(image, low_threshold, high_threshold)

# 高斯滤波
def gaussian_blur(image, kernel_size):
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

# 生成兴趣区域即 Mask 掩模
def region_of_interest(image, vertices):
    mask = np.zeros_like(image)  # 创建与输入图像大小相同的零矩阵，作为掩模
    if len(image.shape) > 2:  # 如果输入是彩色图像
        ignore_mask_color = (255,) * image.shape[2]  # 设置填充颜色为白色（多通道）
      
    else:  # 如果输入是灰度图像
        ignore_mask_color = 255  # 设置填充颜色为白色（单通道）
    
    # 用白色填充多边形区域，生成掩模
    cv.fillPoly(mask, vertices, ignore_mask_color)  
    masked_image = cv.bitwise_and(image, mask)  # 与原图像进行位与运算，保留感兴趣区域
    return masked_image

# 霍夫变换检测直线
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    # rho: 距离精度，theta: 角度精度，threshold: 累加阈值
    # min_line_len: 最小线段长度，max_line_gap: 最大断裂长度
    lines = cv.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines  # 返回检测到的线段，每条线段由起点和终点坐标表示

# 绘制车道线
def draw_lines(image, lines, color=[255,0,0], thickness=2):
    right_y_set, right_x_set, right_slope_set = [], [], []  # 存储右车道线的 y 坐标、x 坐标和斜率
    left_y_set, left_x_set, left_slope_set = [], [], []  # 存储左车道线的 y 坐标、x 坐标和斜率
    slope_min, slope_max = 0.35, 0.85  # 定义车道线斜率的有效范围
    middle_x = image.shape[1] / 2  # 计算图像水平中线位置
    max_y = image.shape[0]  # 获取图像高度
    for line in lines:  # 遍历所有检测到的线段
        for x1, y1, x2, y2 in line:  # 提取线段的起点 (x1, y1) 和终点 (x2, y2)
            fit = np.polyfit((x1, x2), (y1, y2), 1)  # 对线段进行一次多项式拟合
            slope = fit[0]  # 获取斜率
            if slope_min < np.absolute(slope) <= slope_max:  # 检查斜率是否在有效范围内
                if slope > 0 and x1 > middle_x and x2 > middle_x:  # 判断为右车道线（正斜率且在图像右侧）
                    right_y_set.extend([y1, y2])
                    right_x_set.extend([x1, x2])
                    right_slope_set.append(slope)
                elif slope < 0 and x1 < middle_x and x2 < middle_x:  # 判断为左车道线（负斜率且在图像左侧）
                    left_y_set.extend([y1, y2])
                    left_x_set.extend([x1, x2])
                    left_slope_set.append(slope)
    # 绘制左车道线
    if left_y_set:  # 如果检测到左车道线数据
        lindex = left_y_set.index(min(left_y_set))  # 找到最高点（y 坐标最小）的索引
        left_x_top, left_y_top = left_x_set[lindex], left_y_set[lindex]  # 获取顶部坐标
        lslope = np.median(left_slope_set)  # 使用斜率中值提高稳定性
        left_x_bottom = int(left_x_top + (max_y - left_y_top) / lslope)  # 根据斜率计算底部 x 坐标
        cv.line(image, (left_x_bottom, max_y), (left_x_top, left_y_top), color, thickness)  # 绘制左车道线
    # 绘制右车道线
    if right_y_set:  # 如果检测到右车道线数据
        rindex = right_y_set.index(min(right_y_set))  # 找到最高点（y 坐标最小）的索引
        right_x_top, right_y_top = right_x_set[rindex], right_y_set[rindex]  # 获取顶部坐标
        rslope = np.median(right_slope_set)  # 使用斜率中值提高稳定性
        right_x_bottom = int(right_x_top + (max_y - right_y_top) / rslope)  # 根据斜率计算底部 x 坐标
        cv.line(image, (right_x_bottom, max_y), (right_x_top, right_y_top), color, thickness)  # 绘制右车道线
    return image  # 返回绘制车道线的图像

# 图像融合
def weighted_img(img, initial_img, a=0.8, b=1., c=0.):
    return cv2.addWeighted(initial_img, a, img, b, c)
# 主处理函数
def process_image(image):
    rho = 1  # 霍夫变换的距离精度（像素）
    theta = np.pi / 180  # 霍夫变换的角度精度（1度）
    hof_threshold = 20  # 霍夫变换的累加阈值
    min_line_len = 30  # 霍夫变换检测的最小线段长度
    max_line_gap = 60  # 霍夫变换允许的最大线段断裂长度
    kernel_size = 5  # 高斯模糊的核大小
    canny_low_threshold = 75  # Canny 边缘检测的低阈值
    canny_high_threshold = canny_low_threshold * 3  # Canny 边缘检测的高阈值（低阈值的 3 倍）
    alpha, beta, lambda_ = 0.8, 1., 0.  # 图像融合的权重参数
    imshape = image.shape  # 获取输入图像的尺寸（高度、宽度、通道数）
    # 灰度图转换
    gray = grayscale(image)
    # 高斯滤波
    blur_gray = gaussian_blur(gray, kernel_size)
    # Canny 边缘检测
    edge_image = canny(blur_gray, canny_low_threshold, canny_high_threshold)
    # 生成掩模区域
    vertices = np.array([[(0, imshape[0]), (9 * imshape[1] / 20, 11 * imshape[0] / 18),
                          (11 * imshape[1] / 20, 11 * imshape[0] / 18), (imshape[1], imshape[0])]], dtype=np.int32)
    
    masked_edges = region_of_interest(edge_image, vertices)  # 提取感兴趣区域，限制检测范围
    # 霍夫变换检测直线
    lines = hough_lines(masked_edges, rho, theta, hof_threshold, min_line_len, max_line_gap)
    line_image = np.zeros_like(image)  # 创建与原图大小相同的空白图像，用于绘制车道线
    # 绘制车道线
    line_image = draw_lines(line_image, lines)
    # 图像融合
    lines_edges = weighted_img(line_image, image, alpha, beta, lambda_)

    return lines_edges  # 返回处理后的图像

if __name__ == '__main__':
    image = cv.imread('jupyter/test1.png')  
    line_image = process_image(image)  # 调用主函数处理图像
    cv.imwrite('result/task8/resultline_img.jpg', line_image)  

### 附加实验1: SIFT 特征匹配与图像拼接

In [79]:
import cv2 as cv
import numpy as np 

def get_homo(img1, img2):
    """计算两幅图像之间的单应性矩阵"""
    sift = cv.SIFT_create()  # 创建 SIFT 特征检测器，用于提取图像特征点
    
    # 提取图像的关键点和描述子
    k1, des1 = sift.detectAndCompute(img1, None)
    k2, des2 = sift.detectAndCompute(img2, None)
    
    # 在图像上绘制关键点并保存
    img1_keypoints = cv.drawKeypoints(img1, k1, None)
    cv.imwrite("result/task_add_1/sift_keypoints.jpg", img1_keypoints)

    # 创建暴力匹配器，用于匹配两幅图像的特征点
    bf = cv.BFMatcher()
    
    # 使用 KNN 算法匹配描述子，k=2 表示每个点找两个最近邻
    matches = bf.knnMatch(des1, des2, k=2)
    
    # 提取所有匹配对的第一个邻居
    all_matches = [m[0] for m in matches]
    
    # 绘制所有匹配并保存结果
    match_img = cv.drawMatches(img1, k1, img2, k2, all_matches, None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv.imwrite("result/task_add_1/sift_matches.jpg", match_img)

    # 筛选优质匹配，使用 Lowe's ratio test
    verify_ratio = 0.8  # 匹配比率阈值，判断匹配质量
    verify_matches = []
    for m1, m2 in matches:
        if m1.distance < verify_ratio * m2.distance:  # 如果第一个邻居距离小于第二个的 0.8 倍，则保留
            verify_matches.append(m1)
    
    # 绘制优质匹配并保存
    good_matches_img = cv.drawMatches(img1, k1, img2, k2, verify_matches, None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv.imwrite("result/task_add_1/sift_good_matches.jpg", good_matches_img) 
    
    # 检查匹配点数是否足够计算单应性矩阵
    min_matches = 8  # 最小匹配点数要求
    if len(verify_matches) > min_matches:
        # 提取匹配点的坐标
        img1_pts = np.float32([k1[m.queryIdx].pt for m in verify_matches]).reshape(-1, 2)
        img2_pts = np.float32([k2[m.trainIdx].pt for m in verify_matches]).reshape(-1, 2)

        # 使用 RANSAC 算法计算单应性矩阵
        H, _ = cv.findHomography(img1_pts, img2_pts, cv.RANSAC, 5.0)
        return H 
    else:
        print(f"Error: Insufficient matches ({len(verify_matches)} < {min_matches})")  # 输出错误信息
        exit()

def stitch_image(img1, img2, H):
    """根据单应性矩阵拼接两幅图像"""
    h1, w1 = img1.shape[:2]  
    h2, w2 = img2.shape[:2]
    
    # 定义两幅图像的角点坐标
    img1_dims = np.float32([[0,0], [0,h1-1], [w1-1,h1-1], [w1-1,0]]).reshape(-1,1,2) 
    img2_dims = np.float32([[0,0], [0,h2-1], [w2-1,h2-1], [w2-1,0]]).reshape(-1,1,2)
    
    # 对第一幅图像的角点应用单应性变换
    img1_transform = cv.perspectiveTransform(img1_dims, H)
    result = np.concatenate([img2_dims, img1_transform], axis=0)  # 合并两幅图像的角点坐标
    
    [x_min, y_min] = np.int32(np.floor(result.min(axis=0).ravel())) 
    [x_max, y_max] = np.int32(np.ceil(result.max(axis=0).ravel())) 
    transform_dist = [-x_min, -y_min]  # 计算平移距离以避免负坐标
    
    # 创建平移矩阵，用于调整图像位置
    transform_array = np.array([[1,0,transform_dist[0]], [0,1,transform_dist[1]], [0,0,1]])
    
    # 组合单应性矩阵和平移矩阵
    output_shape = (x_max-x_min, y_max-y_min)
    H_output = transform_array @ H
    
    # 对第一幅图像应用透视变换并拼接第二幅图像
    result_img = cv.warpPerspective(img1, H_output, output_shape)
    result_img[transform_dist[1]:transform_dist[1]+h2, transform_dist[0]:transform_dist[0]+w2] = img2
    return result_img  # 返回拼接后的图像

# 主程序
images = []
for i in range(3):
    filename = 'jupyter/image{}.jpg'.format(i+1)
    img = cv.imread(filename)
    img = cv.resize(img,(640,480))
    images.append(img)

for i in range(1, len(images)):
    H = get_homo(images[i], images[i-1])
    images[i] = stitch_image(images[i], images[i-1], H)
    
    
res = images[len(images)-1]
cv.imwrite("result/task_add_1/stitched_image.jpg", res)

True

### 附加实验2: 细胞轮廓识别与编号标注

In [85]:
import cv2  
img = cv2.imread('jupyter/count.jpg', 1)

# 将彩色图像转换为灰度图像
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

# 对灰度图像进行阈值处理，150 为阈值，255 为最大值，THRESH_BINARY_INV 表示反转二值化
_, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

# 创建一个 5x5 的椭圆形结构元素，用于形态学操作
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

# 使用腐蚀操作去除图像中的噪声
eroded = cv2.erode(binary, kernel, iterations=4)

# 对腐蚀后的图像进行膨胀
dilated = cv2.dilate(eroded, kernel, iterations=2)

# 使用 3x3 高斯核对图像进行模糊处理，平滑边缘，减少噪声影响
gaussian = cv2.GaussianBlur(dilated, (3, 3), 0)

# 检测图像中的轮廓
contours, _ = cv2.findContours(gaussian, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

contoursOK = [] 
for i in contours:  # 遍历所有检测到的轮廓
    if cv2.contourArea(i) > 30:  # 如果轮廓面积大于 30，则认为是有意义的轮廓
        contoursOK.append(i) 

# 在原图像上绘制绿色轮廓
draw = cv2.drawContours(img, contoursOK, -1, (0, 255, 0), 2)

# 遍历筛选后的轮廓及其索引
for i, j in zip(contoursOK, range(len(contoursOK))):
    M = cv2.moments(i)  # 计算当前轮廓的矩，用于后续质心计算
    cX = int(M["m10"] / M["m00"])  
    cY = int(M["m01"] / M["m00"])
    
    # 在质心位置标注轮廓的编号
    cv2.putText(img, str(j), (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2)

# 保存高斯模糊处理后的图像
cv2.imwrite("result/task_add_2/gaussian.jpg", gaussian)

# 保存结果图像 'draw.jpg'
cv2.imwrite("result/task_add_2/draw.jpg", draw)

True